In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [2]:
train_dir = r'C:\Users\JAYANTH\Lung Cancer Detection\LungCT_dataset\train'
val_dir   = r'C:\Users\JAYANTH\Lung Cancer Detection\LungCT_dataset\val' 

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 788 images belonging to 3 classes.
Found 162 images belonging to 3 classes.


In [6]:
from tensorflow import keras
from keras import models,layers,optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [7]:
model = models.Sequential()
    
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [9]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam', metrics = ['acc'])

In [10]:
train_size = len(train_generator)
val_size = len(val_generator)

In [11]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight('balanced',classes =  np.unique(train_generator.labels),y =  train_generator.labels)
class_weights_dict = dict(enumerate(class_weights))

In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_size,
    epochs = 8,
    validation_data = val_generator,
    validation_steps = val_size,
    class_weight=class_weights_dict
)

Epoch 1/8
25/25 [==============================] - 13s 263ms/step - loss: 1.3256 - acc: 0.3312 - val_loss: 1.0960 - val_acc: 0.5000
Epoch 2/8
25/25 [==============================] - 11s 427ms/step - loss: 1.0991 - acc: 0.3388 - val_loss: 1.0994 - val_acc: 0.0926
Epoch 3/8
25/25 [==============================] - 11s 431ms/step - loss: 1.0988 - acc: 0.3794 - val_loss: 1.0947 - val_acc: 0.4877
Epoch 4/8
25/25 [==============================] - 11s 432ms/step - loss: 1.0903 - acc: 0.4365 - val_loss: 1.0406 - val_acc: 0.4506
Epoch 5/8
25/25 [==============================] - 11s 440ms/step - loss: 1.0727 - acc: 0.2398 - val_loss: 1.0498 - val_acc: 0.1358
Epoch 6/8
25/25 [==============================] - 11s 435ms/step - loss: 1.0679 - acc: 0.1624 - val_loss: 1.0552 - val_acc: 0.1358
Epoch 7/8
25/25 [==============================] - 11s 432ms/step - loss: 1.0671 - acc: 0.2043 - val_loss: 1.0571 - val_acc: 0.1358
Epoch 8/8
25/25 [==============================] - 11s 434ms/step - loss: 1.

In [13]:
test_dir = r'C:\Users\JAYANTH\Lung Cancer Detection\LungCT_dataset\test'
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size = (224,224),
        batch_size = 32,
        class_mode='binary'
)

Found 147 images belonging to 3 classes.


In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Generate predictions for test set
predictions = model.predict(test_generator)

# Convert labels and predictions to binary format
binary_test_labels = test_generator.labels
binary_predictions = np.argmax(predictions, axis=1)

# Calculate precision and recall
precision = precision_score(binary_test_labels, binary_predictions, average='macro')
recall = recall_score(binary_test_labels, binary_predictions, average='macro')
f1 = f1_score(binary_test_labels, binary_predictions, average='macro')

print('Precision:',precision)
print('Recall:',recall)
print('F1 score:',f1)

5/5 [==============================] - 1s 300ms/step
Precision: 0.16576168929110105
Recall: 0.30210060807075734
F1 score: 0.21194102006284746


C:\Users\JAYANTH\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
